In [1]:
import pandas as pd
import numpy as np

In [2]:
dfinit= pd.read_csv("dfinit.csv")
dfup1= pd.read_csv("dfup1.csv")
dfup2= pd.read_csv("dfup2.csv")

# work with dfup2 file 

In [3]:
dfup2_1=dfup2[dfup2["form_field"]==1]

In [4]:
dfup2_2=dfup2[dfup2["form_field"]==2]

In [5]:
dfup2_1_drop=dfup2_1.drop(columns="form_field").set_index("recordid")
dfup2_2_drop=dfup2_2.drop(columns="form_field").set_index("recordid")

In [6]:
dfup2_merge= dfup2_1_drop.combine_first(dfup2_2_drop)

In [7]:
dfup2=dfup2_merge.fillna(int(-1))


# work with dfinit file 

We fill the null values with -1 and set 'recordid' as the index.

In [8]:
dfinit=dfinit.fillna(-1).set_index("recordid")

# work with dfup1 file 

We fill the null values with -1 and set 'recordid' as the index.

In [9]:
dfup1=dfup1.fillna(-1).set_index("recordid")

# work with mapping_file

We utilize the mapping_file to rename columns in the dfinit and dfup1 datasets. Initially, we extract rows from mapping_file for individual columns ("dfinit", "dfup1", "dfup2"). We then create three lists: mapping_file_dfinit, mapping_file_dfup1, and mapping_file_dfup2. Column names from the dfinit and dfup1 datasets are also extracted. We adopt the dfup2 column names from the mapping_file dataset as our standard column naming convention.

Next, we employ a for loop to iterate over the column names in the dfup1_col list. Each column name is split using split("_"), yielding two distinct pieces of data: the field name and the subfield number. We also determine the index number from mapping_file_dfup2 using the field name obtained from the split. With both the index number and subfield number in hand, we construct a new column name using the index_mapping_file number and the subfield number (from the second split value). Subsequently, we rename the column in the dfup1 dataset.

The same procedure is applied to the dfinit dataset.


In [10]:
mapping_file= pd.read_csv("mapping_file.csv")

In [11]:
dfint_col=dfinit.columns
dfup1_col=dfup1.columns
mapping_file_dfinit=[]
mapping_file_dfup1=[]
mapping_file_dfup2=[]
for _, row in mapping_file.iterrows():
    mapping_file_dfinit.append(row["dfinit"])
    mapping_file_dfup1.append(row["dfup1"])
    mapping_file_dfup2.append(row["dfup2"])
    
# print(mapping_file_dfinit)
    
for col in dfup1_col:
    x=col.split("_")
    if x[0] in mapping_file_dfup1:
        index_mapping_file=mapping_file_dfup1.index(x[0])
        new_col_name=f"{mapping_file_dfup2[index_mapping_file]}_{x[1]}"
        dfup1.rename(columns={col :new_col_name}, inplace=True)
for col in dfint_col:
    x=col.split("_")
    if x[0] in mapping_file_dfinit:
        index_mapping_file=mapping_file_dfinit.index(x[0])
        new_col_name=f"{mapping_file_dfup2[index_mapping_file]}_{x[1]}"
        dfinit.rename(columns={col :new_col_name}, inplace=True)


# work with field_index
    

The field_index dataset provides information about which columns are present in each of our three main datasets: dfinit, dfup1, and dfup2. We only consider columns that are available in all three datasets. First, we establish a logic, then drop an unnecessary row "recordid". This selection is subsequently applied to the dfinit, dfup1, and dfup2 datasets. 
Finally, we concatenate the three datasets to obtain our final consolidated dataset.

In [12]:
fild_index= pd.read_csv("field_index.csv")

In [13]:
fild_index = fild_index[(fild_index["dfinit"]==1) & (fild_index["dfup1"]==1) & (fild_index["dfup2"]==1)]

In [14]:
fild_index

,Code,dfinit,dfup1,dfup2
0,recordid,1,1,1
1,mvl_0,1,1,1
2,hik_0,1,1,1
3,aom_0,1,1,1
4,aom_1,1,1,1
...,...,...,...,...
99,odr_4,1,1,1
100,ueu_0,1,1,1
101,ueu_1,1,1,1
102,ueu_2,1,1,1


In [15]:
fild_index= fild_index.drop(0)


In [16]:
fild_index_list=[]
for _, row in fild_index.iterrows():
    fild_index_list.append(row["Code"].upper())
dfinit= dfinit[fild_index_list]
dfup1= dfup1[fild_index_list]
dfup2=dfup2[fild_index_list]

In [17]:
final_data=pd.concat([dfinit,dfup1,dfup2],ignore_index=False)